In [3]:
#creates a map with mean temp and study data
import netCDF4 as nc
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objs as go

# Open the NetCDF file
data = nc.Dataset('/data/cru_ts4.06.2021.2021.tmp.dat.nc')

# Extract the necessary variables
lon = data.variables['lon'][:]
lat = data.variables['lat'][:]
temperature = data.variables['tmp'][:]

# Calculate the mean temperature per year
mean_temperature_yearly = np.mean(temperature, axis=0)

# Create a DataFrame for the mean temperature data
mean_temperature_df = pd.DataFrame({
    'longitude': np.repeat(lon, len(lat)),
    'latitude': np.tile(lat, len(lon)),
    'mean_temperature': mean_temperature_yearly.flatten(order='F')
})

# Remove null values
mean_temperature_df = mean_temperature_df.dropna(subset=['mean_temperature'])

# Create the world map using Plotly
fig = px.scatter_geo(
    data_frame=mean_temperature_df,
    lon='longitude',
    lat='latitude',
    hover_name='mean_temperature',
    color='mean_temperature',
    projection='natural earth',
    color_continuous_scale='jet',
    color_continuous_midpoint=np.median(mean_temperature_df['mean_temperature']),
)
# Reduce the size and opacity of the climate zone markers
fig.update_traces(marker=dict(size=3, opacity=0.3))  # Adjust the size and opacity as needed




# Read the country study data from the Excel file
country_study_data = pd.read_excel('/data/country_study_data.xlsx')
# Remove trailing spaces from column names
country_study_data.columns = country_study_data.columns.str.strip()
# Define the color scale range based on the income classifications
color_scale_range = ['Red', 'Orange', 'Green', 'Blue']

# Map the income classifications to color scale values
color_map = {
    'LIC': 'rgb(255, 0, 0)',       # Red
    'LMIC': 'rgb(255, 165, 0)',    # Orange
    'UMIC': 'rgb(0, 255, 0)',    # Green
    'HIC': 'rgb(0, 0, 255)'        # Blue
}
# Create a Choropleth trace with country data
country_trace = go.Choropleth(
    locations=country_study_data['Country'],
    locationmode='country names',
    z=country_study_data['Number of Studies'],
    colorscale='Greys',
    autocolorscale=False,
    zmin=country_study_data['Number of Studies'].min()-1,
    zmax=country_study_data['Number of Studies'].max(),
    text=country_study_data['Country'] + ': ' + country_study_data['Number of Studies'].astype(str) + ' studies',  # Add labels with country name and number of studies
    marker=dict(
        line=dict(
            color=country_study_data['Classification by Income'].map(color_map),
            width=3
        )
    ),
    colorbar=dict(
        title='Number of Studies(shading)',
        x=0,  # Specify the x position of the colorbar
        y=0.25,  # Specify the y position of the colorbar
        len=0.5,  # Specify the length of the colorbar (fraction of the plot height)
        thickness=20,  # Specify the thickness of the colorbar (in pixels)
        titleside='right',  # Position the colorbar title on the right side
        titlefont=dict(size=12),  # Customize the font size of the colorbar title
        tickfont=dict(size=10),  # Customize the font size of the colorbar tick labels
        tickvals=[0, 1, 2, 3, 4, 5],  # Specify the tick values
        ticktext=[0, 1, 2, 3, 4, 5],  # Specify the tick labels
        tickcolor='white',  # Set tickcolor to make ticks visible against the bar
        ticklen=8,  # Set ticklen to adjust the length of the ticks
        tickwidth=1,  # Set tickwidth to adjust the width of the ticks
    )
)
# Create a Choropleth trace with GDP Data for legend
gdp_trace = go.Choropleth(
    locations=country_study_data['Country'],
    locationmode='country names',
    z=country_study_data['Classification by Income'],
    colorscale=color_scale_range,
    autocolorscale=False,
    zmin=country_study_data['Number of Studies'].min()-1,
    zmax=country_study_data['Number of Studies'].max(),
    colorbar=dict(
        title='Classification by GDP(outline)',
        x=0,  # Specify the x position of the colorbar
        y=0.75,  # Specify the y position of the colorbar
        len=0.5,  # Specify the length of the colorbar (fraction of the plot height)
        thickness=20,  # Specify the thickness of the colorbar (in pixels)
        titleside='right',  # Position the colorbar title on the bottom
        titlefont=dict(size=12),  # Customize the font size of the colorbar title
        tickfont=dict(size=10),  # Customize the font size of the colorbar tick labels
        tickvals=[0, 1, 2, 3],  # Specify the tick values
        ticktext=list(color_map.keys()),  # Specify the tick labels
        tickmode='array',
        tickcolor='white',  # Set tickcolor to make ticks visible against the bar
        ticklen=8,  # Set ticklen to adjust the length of the ticks
        tickwidth=1,  # Set tickwidth to adjust the width of the ticks
    )
)
# Add the country trace and color axis to the figure
fig.add_trace(country_trace)
# Add the gdp trace to the figure
fig.add_trace(gdp_trace)
# Update the layout with editable parameters
fig.update_layout(
    title='Overview of Study Locations',  # Set the title
    title_font=dict(size=24),  # Customize the title font size
    margin=dict(l=0, r=0, t=100, b=0),  # Adjust the margins
    title_x=0.5,
    legend=dict(
        title='Legend',  # Set the legend title
        font=dict(size=12),  # Customize the legend font size
        bgcolor='rgba(0, 0, 0, 0.5)',  # Set the legend background color
        bordercolor='rgb(0, 0, 0)',  # Set the legend border color
        borderwidth=1,  # Set the legend border width
    ),
    coloraxis=dict(colorbar=dict(
            title='Annual Mean Temperature(celsius)',
            len=0.5,  # Specify the length of the colorbar (fraction of the plot height)
            thickness=20,  # Specify the thickness of the colorbar (in pixels)
            titleside='right',  # Position the colorbar title on the right side
            titlefont=dict(size=12),  # Customize the font size of the colorbar title
            tickfont=dict(size=10),  # Customize the font size of the colorbar tick labels)),
            ),
            geo=dict(
                showframe=False,  # Hide the map frame
                showcoastlines=True,  # Show coastlines
                coastlinecolor='rgb(50, 50, 50)',  # Customize the coastline color
                showocean=False,  # Show ocean color
                oceancolor='rgb(0, 0, 50)',  # Customize the ocean color
                showland=False,  # Show land color
                landcolor='rgb(240, 240, 240)',  # Customize the land color
                showcountries=True,
                showlakes=True,
             )
    )         
)
# Save the updated map as an HTML file
fig.write_html('output/mean_temp_map_with_study_data.html', auto_open=True)
#write map to .jpeg file
fig.write_image('output/mean_temp_map_with_study_data.jpeg')



FileNotFoundError: [Errno 2] No such file or directory: '/data/cru_ts4.06.2021.2021.tmp.dat.nc'